In [64]:
import pandas as pd
import os
from PIL import ImageFilter, Image
import random


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import random
import os
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings('ignore')
from functools import partial

import imgaug.augmenters as iaa
import numpy as np
from PIL import ImageFilter, Image
from timm.data import auto_augment
from tqdm.auto import tqdm
import random

_OP_CACHE = {}


def _get_op(key, factory):
    try:
        op = _OP_CACHE[key]
    except KeyError:
        op = factory()
        _OP_CACHE[key] = op
    return op


def _get_param(level, img, max_dim_factor, min_level=1):
    max_level = max(min_level, max_dim_factor * max(img.size))
    return round(min(level, max_level))


def gaussian_blur(img, radius, **__):
    radius = _get_param(radius, img, 0.1)
    key = 'gaussian_blur_' + str(radius)
    op = _get_op(key, lambda: ImageFilter.GaussianBlur(radius))
    return img.filter(op)


def motion_blur(img, k, **__):
    k = _get_param(k, img, 5, 20) | 1  # bin to odd values
    key = 'motion_blur_' + str(k)
    op = _get_op(key, lambda: iaa.MotionBlur(k))
    return Image.fromarray(op(image=np.asarray(img)))


def gaussian_noise(img, scale, **_):
    scale = _get_param(scale, img, 0.25) | 1  # bin to odd values
    key = 'gaussian_noise_' + str(scale)
    op = _get_op(key, lambda: iaa.AdditiveGaussianNoise(scale=scale))
    return Image.fromarray(op(image=np.asarray(img)))


def poisson_noise(img, lam, **_):
    lam = _get_param(lam, img, 0.2) | 1  # bin to odd values
    key = 'poisson_noise_' + str(lam)
    op = _get_op(key, lambda: iaa.AdditivePoissonNoise(lam))
    return Image.fromarray(op(image=np.asarray(img)))


def _level_to_arg(level, _hparams, max):
    level = max * level / auto_augment._LEVEL_DENOM
    return level,


_RAND_TRANSFORMS = auto_augment._RAND_INCREASING_TRANSFORMS.copy()
_RAND_TRANSFORMS.remove('SharpnessIncreasing')  # remove, interferes with *blur ops
_RAND_TRANSFORMS.extend([
    'GaussianBlur',
    'MotionBlur',
    'GaussianNoise',
    'PoissonNoise'
])
auto_augment.LEVEL_TO_ARG.update({
    'GaussianBlur': partial(_level_to_arg, max=4),
    'MotionBlur': partial(_level_to_arg, max=20),
    'GaussianNoise': partial(_level_to_arg, max=0.1 * 255),
    'PoissonNoise': partial(_level_to_arg, max=40)
})
auto_augment.NAME_TO_OP.update({
    'GaussianBlur': gaussian_blur,
    'MotionBlur': motion_blur,
    'GaussianNoise': gaussian_noise,
    'PoissonNoise': poisson_noise
})


def rand_augment_transform(magnitude=5, num_layers=3):
    # These are tuned for magnitude=5, which means that effective magnitudes are half of these values.
    hparams = {
        'rotate_deg': 30,
        'shear_x_pct': 0.9,
        'shear_y_pct': 0.2,
        'translate_x_pct': 0.10,
        'translate_y_pct': 0.30
    }
    ra_ops = auto_augment.rand_augment_ops(magnitude, hparams, transforms=_RAND_TRANSFORMS)
    # Supply weights to disable replacement in random selection (i.e. avoid applying the same op twice)
    choice_weights = [1. / len(ra_ops) for _ in range(len(ra_ops))]
    return auto_augment.RandAugment(ra_ops, num_layers, choice_weights)

In [65]:
hparams = {
        'rotate_deg': 45,
        'shear_x_pct': 0.9,
        'shear_y_pct': 0.2,
        'translate_x_pct': 0.10,
        'translate_y_pct': 0.30
    }

In [66]:
ra_ops = auto_augment.rand_augment_ops(5, hparams, transforms=_RAND_TRANSFORMS)

In [67]:
choice_weights = [1. / len(ra_ops) for _ in range(len(ra_ops))]

In [68]:
tfm=auto_augment.RandAugment(ra_ops, 3, choice_weights)

In [69]:
import pandas as pd
df = pd.read_csv('./synthtiger/results/images/data/train.csv')
df['len'] = df['label'].str.len()

df['len'] = df['label'].str.len()
train_v1 = df[df['len']==1]

train_v1.reset_index(drop=True,inplace=True)

In [70]:
id_list=list(train_v1.id)

In [ ]:
#len이 1인 단어만 augmentation
aug_df=pd.DataFrame()
idx = 200000
for i in tqdm(range(40000)):
    select=random.randint(0,23702)
    id_key=id_list[select]
    imgFile1 = f'./synthtiger/results/images/data/train/{id_key}.png'

    new_data = {
        'id' : f"aug_TRAIN_one_{idx:06d}",
        'img_path' : f"./Cut_mix_one/TRAIN_one_{idx:06d}.png",
        'label' : train_v1['label'][select],
        'len' : int(train_v1['len'][select])
    }
    aug_df=aug_df.append(new_data,ignore_index=True)
    cut_mix_img=Image.open(imgFile1)
    aug_img=tfm(cut_mix_img)
    aug_img.save(f'./synthtiger/results/images/data/Cut_mix_one/TRAIN_one_{idx:06d}.png')
    idx += 1
    print(idx)

In [ ]:
#len이 2인 단어만 augentation하기 위한 코드

os.makedirs("./synthtiger/results/images/data/Cut_mix_two_new", exist_ok=True)
aug_df=pd.DataFrame()
idx = 0
for i in tqdm(range(30000)):
    
    select1, select2 = random.randint(0, 23702), random.randint(0, 23702)
    id_key1=id_list[select1]
    id_key2=id_list[select2]
    imgFile1 = f'./synthtiger/results/images/data/train/{id_key1}.png'
    imgFile2 = f'./synthtiger/results/images/data/train/{id_key2}.png'

    img1 = cv2.imread(imgFile1, 1);
    img2 = cv2.imread(imgFile2, 1);
    
    img1 = cv2.resize(img1,(100,125))
    img2 = cv2.resize(img2,(100,125))
    
    addh = cv2.hconcat([img1, img2])
    
    new_data = {
        'id' : f"TRAIN_two{idx:06d}",
        'img_path' : f"./Cut_mix_two_new/TRAIN_two{idx:06d}.png",
        'label' : train_v1['label'][select1] + train_v1['label'][select2],
        'len' : train_v1['len'][select1] + train_v1['len'][select2]
    }
    
    aug_df = aug_df.append(new_data, ignore_index=True)
    cv2.imwrite(f"./synthtiger/results/images/data/Cut_mix_two_new/TRAIN_two{idx:06d}.png", addh)
    new_data = {
        'id' : f"aug_TRAIN_two{idx:06d}",
        'img_path' : f"./Cut_mix_two_new/aug_TRAIN_two{idx:06d}.png",
        'label' : train_v1['label'][select1] + train_v1['label'][select2],
        'len' : train_v1['len'][select1] + train_v1['len'][select2]
    }
    aug_df=aug_df.append(new_data,ignore_index=True)
    cut_mix_img=Image.open(f"./synthtiger/results/images/data/Cut_mix_two_new/TRAIN_two{idx:06d}.png")
    aug_img=tfm(cut_mix_img)
    aug_img.save(f'./synthtiger/results/images/data/Cut_mix_two_new/aug_TRAIN_two{idx:06d}.png')
    idx += 1
    print(idx)
#     new_data = {
#         'id' : f"aug_TRAIN_one_{idx:06d}",
#         'img_path' : f"./Cut_mix/TRAIN_one_{idx:06d}.png",
#         'label' : train_v1['label'][select],
#         'len' : int(train_v1['len'][select])
#     }
#     aug_df=aug_df.append(new_data,ignore_index=True)
#     cut_mix_img=Image.open(imgFile1)
#     aug_img=tfm(cut_mix_img)
#     aug_img.save(f'./synthtiger/results/images/data/Cut_mix/TRAIN_one_{idx:06d}.png')
#     idx += 1
#     print(idx)

In [88]:
df_len_count = pd.DataFrame(df['len'].value_counts())
df_len_count.reset_index(inplace=True)
df_len_count.columns = ['len', 'len_count']

display(df_len_count)

,len,len_count
0,2,28631
1,1,23703
2,3,13514
3,4,9988
4,5,1026
5,6,26


In [ ]:
len2count = {k:v for k,v in zip(df_len_count['len'], df_len_count['len_count'])}

In [ ]:
#전체 train 데이터셋으로 이미지 cut-mix

idx = 0
for i in tqdm(range(300000)):
    select1, select2 = random.randint(0, 76888), random.randint(0, 76888)
    imgFile1 = f'./data/train/TRAIN_{select1:05d}.png'
    imgFile2 = f'./data/train/TRAIN_{select2:05d}.png'
    
    if df['len'][select1] + df['len'][select2] > 6:
        continue
        
    if len2count[df['len'][select1] + df['len'][select2]] > 200000:
        continue
    
    # 이미지 읽기
    img1 = cv2.imread(imgFile1, 1);
    img2 = cv2.imread(imgFile2, 1);
    
    img1 = cv2.resize(img1,(100,125))
    img2 = cv2.resize(img2,(100,125))
    
    addh = cv2.hconcat([img1, img2])
    
    new_data = {
        'id' : f"TRAIN_{idx:06d}",
        'img_path' : f"./Cut_mix/TRAIN_{idx:06d}.png",
        'label' : df['label'][select1] + df['label'][select2],
        'len' : df['len'][select1] + df['len'][select2]
    }
    
    df = df.append(new_data, ignore_index=True)
    cv2.imwrite(f"./data/Cut_mix/TRAIN_{idx:06d}.png", addh)
    new_data = {
        'id' : f"aug_TRAIN_{idx:06d}",
        'img_path' : f"./Cut_mix/aug_TRAIN_{idx:06d}.png",
        'label' : df['label'][select1] + df['label'][select2],
        'len' : df['len'][select1] + df['len'][select2]
    }
    df=df.append(new_data,ignore_index=True)
    cut_mix_img=Image.open(f"./data/Cut_mix/TRAIN_{idx:06d}.png")
    aug_img=tfm(cut_mix_img)
    aug_img.save(f'./data/Cut_mix/aug_TRAIN_{idx:06d}.png')
    idx += 1
    len2count[df['len'][select1] + df['len'][select2]] += 1
    print(idx)